# Test 4: Signorini Switching

Interactive exploration of bidirectional Signorini switching under
time-varying rain. Adjust rain schedule, column height, and ramp duration.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from nitsche_signorini import SAND
from nitsche_signorini.runners import make_piecewise_linear, run_splitting_ramp

In [ ]:
soil = SAND
H = 2.0
maxh = 0.05
dt_macro = 60.0
t_final = 18000.0  # 5 hours

# Adjustable rain schedule
p_high = 10.0 * soil.K_s
p_func = make_piecewise_linear([
    (0, 0), (1800, p_high), (5400, p_high), (7200, 0), (99999, 0)])

snap_times = np.array(
    [900, 1800, 3600, 5400, 6300, 7200, 10800, 14400, 18000], dtype=float)

In [ ]:
z, snaps, info = run_splitting_ramp(
    soil, H, maxh, dt_macro, t_final, p_func, snap_times)

nits = info['newton_its']
n_seepage = sum(1 for m in info['signorini_mode'] if m == 'seepage')
n_infilt = sum(1 for m in info['signorini_mode'] if m == 'infiltration')
print(f"Steps: {info['n_steps']}, Newton: mean={nits.mean():.1f}, max={nits.max()}")
print(f"Mode: {n_seepage} seepage, {n_infilt} infiltration")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
cmap = plt.cm.viridis

# (a) Rain + Newton
ax = axes[0, 0]
t_hist = np.array(info['t_history']) / 3600
p_hist = np.array(info['p_rain_history'])
ax.plot(t_hist, p_hist / soil.K_s, 'b-', lw=1.5)
ax.set(xlabel='Time [h]', ylabel='p / K_s', title='Rain schedule')

# (b) theta(z)
ax = axes[0, 1]
for i, (t_s, th_s) in enumerate(snaps):
    c = cmap(i / max(len(snaps) - 1, 1))
    label = f't={t_s/3600:.2f}h' if t_s > 0 else 'IC'
    ax.plot(th_s, z, color=c, lw=1.2, label=label)
ax.set(xlabel='theta [-]', ylabel='z [m]', title='theta(z)')
ax.legend(fontsize=5)

# (c) h_top(t)
ax = axes[1, 0]
t_steps = np.array(info['t_history'][1:]) / 3600
ax.plot(t_steps, info['h_top_history'], lw=1)
ax.axhline(0, color='grey', ls=':', lw=0.5)
ax.set(xlabel='Time [h]', ylabel='h_top [m]', title='h at top boundary')

# (d) Mode
ax = axes[1, 1]
modes = info['signorini_mode']
mode_num = np.array([0 if m == 'seepage' else 1 for m in modes])
changes = np.where(np.diff(mode_num) != 0)[0] + 1
boundaries = np.concatenate([[0], changes, [len(mode_num)]])
for k in range(len(boundaries) - 1):
    si = boundaries[k]
    ei = boundaries[k + 1] - 1
    t0 = 0.0 if si == 0 else t_steps[si]
    t1 = t_steps[min(ei, len(t_steps) - 1)]
    color = 'steelblue' if mode_num[si] == 0 else 'sandybrown'
    ax.axvspan(t0, t1, alpha=0.4, color=color)
ax.set_xlim(t_steps[0], t_steps[-1])
ax.set_ylim(0, 1)
ax.set_yticks([])
ax.set(xlabel='Time [h]', title='Signorini mode')
ax.legend(handles=[
    Patch(facecolor='steelblue', alpha=0.4, label='seepage'),
    Patch(facecolor='sandybrown', alpha=0.4, label='infiltration'),
], fontsize=7)

fig.suptitle('Bidirectional Signorini switching')
fig.tight_layout()
plt.show()